In [ ]:
%matplotlib inline
import os
import os.path
import csv
import numpy
import scipy.stats
import scipy.cluster
import itertools
import math
import sys
import sklearn.cluster
import sklearn.decomposition
import sklearn.manifold
import skimage.filters
import pandas
from matplotlib import pylab, mlab
import matplotlib.cm
import IPython.display
# local modules (in same dir as this notebook)
import plotutils
import syspathutils

In [ ]:
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/.env/lib/python2.7/site-packages/')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/lib/')

In [ ]:
import a2audio.rec
import a2pyutils.storage
import a2audio.segmentation
import a2pyutils.convert

In [ ]:
def set_tick_labels(xticks=None, yticks=None):
    if xticks:
        pylab.gca().set_xticklabels(map(lambda x: xticks%x, pylab.gca().get_xticks().tolist()))
    if yticks:
        pylab.gca().set_yticklabels(map(lambda x: yticks%x, pylab.gca().get_yticks().tolist()))

def compute_mask(spectrum, **params):
    S = spectrum
    p = params.get('percentile', 5)
    r = params.get('radius', 10)
    th = params.get('th', 'yen')
    # remove the band medians
    band_medians = numpy.percentile(S, 50, 1).tolist()
    divide_by_median = lambda r, m: r / m.pop(0)
    S_nomed = numpy.apply_along_axis(divide_by_median, 1, S, band_medians)
    # compute the range
    Smin = scipy.ndimage.percentile_filter(S_nomed, p, r)
    Smax = scipy.ndimage.percentile_filter(S_nomed, 100 - p, r)
    Srange = Smax-Smin
    Sdb = a2pyutils.convert.power2decibels(Srange)

    # threshold using yen or a given number
    if th == 'yen':
        th = skimage.filter.threshold_yen(Sdb)
        Smask = Srange > th
    else:
        Smask = Sdb > th

    return Smask, Srange, S_nomed, Sdb, S


In [ ]:
# rec_storage = a2pyutils.storage.from_uri('/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/el_verde1_sample/')
rec_storage = a2pyutils.storage.from_uri('/home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca1_sample/')
# rec = a2audio.rec.Rec('el_verde1-2010-08-12_05-40.flac', '', rec_storage)
rec = a2audio.rec.Rec('sabana_seca1-2008-03-28_22-08.flac', '', rec_storage)
# rec = a2audio.rec.Rec('sabana_seca1-2010-08-05_09-50.flac', '', rec_storage)
rec_spec = rec.getSpectrogram()[0]
rec_duration = len(rec.original) * 1.0 / rec.sample_rate

In [ ]:
Smask, Srange, S_nomed, Sdb, S = compute_mask(rec_spec)
Slabels = scipy.ndimage.measurements.label(Smask)[0]
objs = scipy.ndimage.measurements.find_objects(Slabels)

In [ ]:
sc = rec.original.shape[0] * 1.0 / rec_spec.shape[1]
i0 = 6000
i1 = i0 + 1500
j0, j1 = int(sc * i0), int(sc * i1)
t0, t1 = j0 / rec.sample_rate, j1 / rec.sample_rate
f0, f1 = 0, (rec.sample_rate / 2) / 1000.0
print i0,i1, j0,j1

pylab.figure(figsize=(20,5))
pylab.plot([i / rec.sample_rate for i in xrange(j0, j1)], rec.original[j0:j1])
set_tick_labels('%ds')

for I in [
    a2pyutils.convert.power2decibels(rec_spec),
    a2pyutils.convert.power2decibels(S_nomed),
    a2pyutils.convert.power2decibels(Srange)
]:
    pylab.figure(figsize=(20,5))
    pylab.imshow(I[:, i0:i1], origin="bottom", cmap=matplotlib.cm.jet,
                extent=[t0,t1,f0,f1], aspect='auto')
    pylab.colorbar()
    set_tick_labels('%ds', '%dkHz')

pylab.figure(figsize=(20,5))
pylab.imshow(Smask[:, i0:i1], origin="bottom", cmap=matplotlib.cm.Paired,
        extent=[t0,t1,f0,f1], aspect='auto')
pylab.colorbar()
set_tick_labels('%ds', '%dkHz')
    # pylab.figure(figsize=(20,5))
# pylab.specgram(rec.original[j0:j1])
# pylab.colorbar()
q=1

In [ ]:
# Sdb1  = a2pyutils.convert.power2decibels(Srange)[:, i0:i1]
Sdb1  = Srange[:, i0:i1]
Sdb1[Sdb1 < 0] = 0
Sdb1/=Sdb1.max()

for i in range(0+1, 10+1, 2):
    pylab.figure(figsize=(20,5))
    pylab.imshow(Sdb1**i, origin="bottom", cmap=matplotlib.cm.gist_earth, extent=[t0,t1,f0,f1], aspect='auto')
#     Hx = numpy.arange(Xmin, Xmax, (Xmax - Xmin) / H.shape[1])
#     Hy = numpy.arange(Ymin, Ymax, (Ymax - Ymin) / H.shape[0])
